# BESTLIB Demo - Comunicación Bidireccional

Este notebook demuestra la **comunicación bidireccional** entre JavaScript y Python:
- **Python → JavaScript**: Envío de datos, configuraciones y gráficos
- **JavaScript → Python**: Captura de eventos de interacción (brush, click, zoom)

## Características implementadas:
- ✅ Bar Chart interactivo con brush de selección
- ✅ Scatter Plot con zoom, pan y selección
- ✅ Callbacks Python que reciben eventos del usuario
- ✅ Sistema de comm bidireccional

In [3]:
# Importar BESTLIB
import sys, pathlib
root = pathlib.Path().resolve().parents[0]
sys.path.insert(0, str(root))
from BESTLIB import MatrixLayout

print("✅ BESTLIB importado correctamente")
print(f"📁 Ruta: {root}")

✅ BESTLIB importado correctamente
📁 Ruta: /Users/nahiaescalante/Documents/2025/Visualizacion/bestlib


---

## 📊 Ejemplo 1: Bar Chart Interactivo

Crea un gráfico de barras con **brush de selección**. Cuando arrastres para seleccionar barras, Python recibirá el evento.

In [12]:
# Datos para el bar chart
datos_ventas = [
    {"category": "Ene", "value": 120},
    {"category": "Feb", "value": 190},
    {"category": "Mar", "value": 150},
    {"category": "Abr", "value": 280},
    {"category": "May", "value": 210},
    {"category": "Jun", "value": 340},
    {"category": "Jul", "value": 290},
]

# Configurar el layout con HTML simple
MatrixLayout.set_safe_html(True)
MatrixLayout.map({
    "A": "<h3 style='margin:0; padding:10px; background:#667eea; color:white; text-align:center;'>📊 Ventas 2024</h3>",
    "B": {
        "type": "bar",
        "data": datos_ventas,
        "color": "#4a90e2",
        "hoverColor": "#357abd",
        "interactive": True,  # Habilita el brush
        "axes": True
    }
})

# Crear el layout
layout_bar = MatrixLayout("""
AAA
BBB
BBB
""")

# Registrar callback para capturar selecciones
def on_bar_select(payload):
    print("🎯 Barras seleccionadas:")
    print(f"  - Índices: {payload['indices']}")
    print(f"  - Datos: {payload['items']}")
    if payload['items']:
        total = sum(item['value'] for item in payload['items'])
        print(f"  - Total seleccionado: {total}")

layout_bar.on('select', on_bar_select)

print("✅ Bar chart configurado")
print("👉 Ejecuta la siguiente celda para visualizarlo")
print("💡 Arrastra sobre las barras para seleccionar")

✅ Bar chart configurado
👉 Ejecuta la siguiente celda para visualizarlo
💡 Arrastra sobre las barras para seleccionar


In [13]:
# Renderizar el layout
layout_bar

---

## 🎯 Ejemplo 2: Scatter Plot Interactivo

Crea un scatter plot con **zoom, pan y selección múltiple**. Puedes hacer click en puntos individuales o seleccionar múltiples con brush.

In [6]:
# Generar datos para scatter plot
import random
random.seed(42)

datos_scatter = []
grupos = ["A", "B", "C"]
colores = {"A": "#e24a4a", "B": "#4ae24a", "C": "#4a4ae2"}

for grupo in grupos:
    for i in range(20):
        x = random.gauss(grupos.index(grupo) * 30 + 50, 10)
        y = random.gauss(grupos.index(grupo) * 20 + 40, 8)
        datos_scatter.append({
            "x": x,
            "y": y,
            "label": f"{grupo}{i+1}",
            "color": colores[grupo],
            "group": grupo
        })

print(f"✅ Generados {len(datos_scatter)} puntos en 3 grupos")

✅ Generados 60 puntos en 3 grupos


In [7]:
# Configurar scatter plot
MatrixLayout.map({
    "T": "<h3 style='margin:0; padding:10px; background:#764ba2; color:white; text-align:center;'>🎯 Análisis de Grupos</h3>",
    "S": {
        "type": "scatter",
        "data": datos_scatter,
        "pointRadius": 5,
        "interactive": True,  # Habilita brush y zoom
        "zoom": True,
        "axes": True
    },
    "C": "<div style='padding:10px; font-size:12px;'><b>Controles:</b><br/>🖱️ Click: seleccionar punto<br/>🎨 Brush: selección múltiple<br/>🔍 Scroll: zoom</div>"
})

layout_scatter = MatrixLayout("""
TTTT
SSSC
SSSC
""")

# Callbacks para diferentes eventos
def on_scatter_select(payload):
    print("🎯 Puntos seleccionados con brush:")
    print(f"  - Total: {len(payload['indices'])}")
    if payload['items']:
        grupos = {}
        for item in payload['items']:
            grupo = item.get('group', 'Unknown')
            grupos[grupo] = grupos.get(grupo, 0) + 1
        print(f"  - Por grupo: {grupos}")

def on_point_click(payload):
    print("👆 Click en punto:")
    print(f"  - Índice: {payload['index']}")
    punto = payload['point']
    print(f"  - Label: {punto['label']}")
    print(f"  - Coordenadas: ({punto['x']:.2f}, {punto['y']:.2f})")
    print(f"  - Grupo: {punto['group']}")

layout_scatter.on('select', on_scatter_select)
layout_scatter.on('point_click', on_point_click)

print("✅ Scatter plot configurado")
print("👉 Ejecuta la siguiente celda para visualizarlo")
print("💡 Haz scroll para zoom, arrastra para seleccionar múltiples puntos")

✅ Scatter plot configurado
👉 Ejecuta la siguiente celda para visualizarlo
💡 Haz scroll para zoom, arrastra para seleccionar múltiples puntos


In [8]:
# Renderizar scatter plot
layout_scatter

---

## 🎨 Ejemplo 3: Dashboard Completo

Combinemos ambos gráficos en un dashboard con callbacks globales.

In [9]:
# Configurar callback global
eventos_recibidos = []

def handler_global(payload):
    evento = {
        'tipo': payload.get('type', 'unknown'),
        'num_items': len(payload.get('items', [])),
        'timestamp': len(eventos_recibidos) + 1
    }
    eventos_recibidos.append(evento)
    print(f"📨 Evento #{evento['timestamp']}: {evento['tipo']} con {evento['num_items']} items")

MatrixLayout.on_global('select', handler_global)

# Configurar dashboard
MatrixLayout.map({
    "H": "<h2 style='margin:0; padding:15px; background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); color:white; text-align:center;'>📊 Dashboard Interactivo</h2>",
    "B": {
        "type": "bar",
        "data": datos_ventas,
        "color": "#4a90e2",
        "interactive": True
    },
    "S": {
        "type": "scatter",
        "data": datos_scatter[:30],  # Menos puntos para el dashboard
        "pointRadius": 4,
        "interactive": True,
        "zoom": True
    },
    "I": "<div style='padding:15px; background:#f5f5f5; font-size:13px;'><b>ℹ️ Información:</b><br/>• Interactúa con los gráficos<br/>• Los eventos se capturan en Python<br/>• Usa callbacks para procesar datos</div>"
})

dashboard = MatrixLayout("""
HHHH
BBSS
BBSS
IIIS
""")

print("✅ Dashboard configurado con callback global")
print("👉 Todos los eventos 'select' se procesarán automáticamente")

✅ Dashboard configurado con callback global
👉 Todos los eventos 'select' se procesarán automáticamente


In [10]:
# Renderizar dashboard
dashboard

In [11]:
# Ver resumen de eventos capturados
print("📊 RESUMEN DE EVENTOS CAPTURADOS")
print("=" * 50)
if eventos_recibidos:
    for evt in eventos_recibidos:
        print(f"  #{evt['timestamp']}: {evt['tipo']} → {evt['num_items']} items")
    print("=" * 50)
    print(f"Total de eventos: {len(eventos_recibidos)}")
else:
    print("⏳ No hay eventos todavía")
    print("💡 Interactúa con los gráficos de arriba")

📊 RESUMEN DE EVENTOS CAPTURADOS
⏳ No hay eventos todavía
💡 Interactúa con los gráficos de arriba


---

## 📝 Resumen Técnico

### Comunicación Bidireccional Implementada:

#### Python → JavaScript
```python
MatrixLayout.map({
    "A": {"type": "bar", "data": [...]}  # Envía datos y config
})
```

#### JavaScript → Python
```python
layout.on('select', callback)           # Callback por instancia
MatrixLayout.on_global('select', cb)    # Callback global
```

### Arquitectura:
1. **Python** define datos y callbacks usando `.on()` o `.on_global()`
2. **JavaScript** renderiza con D3.js y captura interacciones del usuario
3. **Jupyter Comm** envía eventos de JS a Python usando `kernel.comm_manager`
4. **Callbacks Python** se ejecutan automáticamente con los datos del evento

### Tipos de eventos disponibles:
- `select`: Selección con brush (barras o puntos)
- `point_click`: Click en punto individual (scatter)
- Extensible: puedes agregar tus propios eventos

---

## 🎯 ¡Listo para tu presentación!

Este sistema demuestra:
- ✅ Comunicación **bidireccional** completa
- ✅ Gráficos **interactivos** profesionales (D3.js)
- ✅ Sistema **escalable** para dashboards complejos
- ✅ Callbacks **flexibles** (por instancia o globales)